<a href="https://colab.research.google.com/github/abosedealli/Stockprediction/blob/main/LSTMAppleAdjusted%20price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
data= pd.read_csv("/content/AAPL.dataset.csv")
data.shape
# create a dataframe with only the Adj close prices
data = data[['Adj Close']]
prices = data

# create a variable to store the number of days you want to forecast
forecast_days = 30

# create a variable to store the percentage of data you want to use for testing
test_data_size = 0.2

# create a variable to store the number of days in your dataset
days_in_dataset = len(data)

# create variables to store the number of days you want to use for testing and training
test_days = int(days_in_dataset * test_data_size)
train_days = days_in_dataset - test_days

# create a training dataset
train_data = data.iloc[:train_days]

# create a testing dataset
test_data = data.iloc[train_days:]

# create a scaler to normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fit the scaler to the training data
scaler.fit(train_data)

# normalize the training data
train_data = scaler.transform(train_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_train = []
y_train = []
for i in range(forecast_days, train_days):
    X_train.append(train_data[i-forecast_days:i])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# create a LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model to the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# normalize the testing data
test_data = scaler.transform(test_data)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Epoch 1/10
265/265 [==============================] - 11s 29ms/step - loss: 7.5203e-04
Epoch 2/10
265/265 [==============================] - 8s 29ms/step - loss: 1.5108e-04
Epoch 3/10
265/265 [==============================] - 8s 29ms/step - loss: 1.0845e-04
Epoch 4/10
265/265 [==============================] - 8s 29ms/step - loss: 1.0894e-04
Epoch 5/10
265/265 [==============================] - 8s 29ms/step - loss: 1.0376e-04
Epoch 6/10
265/265 [==============================] - 8s 29ms/step - loss: 7.7025e-05
Epoch 7/10
265/265 [==============================] - 8s 29ms/step - loss: 7.4490e-05
Epoch 8/10
265/265 [==============================] - 9s 32ms/step - loss: 7.3112e-05
Epoch 9/10
265/265 [==============================] - 8s 29ms/step - loss: 6.1680e-05
Epoch 10/10
265/265 [==============================] - 8s 30ms/step - loss: 5.1352e-05


In [3]:
y_test

array([[1.01606454],
       [1.02257151],
       [1.00660951],
       ...,
       [6.16646575],
       [6.13334264],
       [6.13606497]])

In [2]:
# normalize the testing data
test_data = scaler.transform(test_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_test = []
y_test = []
for i in range(forecast_days, test_days):
    X_test.append(test_data[i-forecast_days:i])
    y_test.append(test_data[i])
X_test, y_test = np.array(X_test), np.array(y_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# make predictions
predictions = model.predict(X_test)

# inverse transform the predictions to get the original prices
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# calculate the mean squared error of the predictions
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)
ape = np.abs((y_test - predictions) / y_test)
# Calculate the mean of the absolute percentage difference
mape = np.mean(ape) * 100
print("MAPE: ", mape)

# Calculate the mean absolute error difference
from sklearn.metrics import mean_absolute_error
mae= np.mean(np.abs(y_test - predictions))
print("MAE: ", mae)

rmse=((mean_squared_error(y_test, predictions))**2)
print("RMSE:", rmse)

import plotly.graph_objs as go

# generate new data
X_forecast = []
for i in range(forecast_days, forecast_days+30):
    X_forecast.append(prices[i-forecast_days:i])
X_forecast = np.array(X_forecast)


forecast = model.predict(X_forecast)

# Get the last predicted value
last_prediction = predictions[-1]

# Add a dimension to the last value of predictions
last_prediction = np.expand_dims(last_prediction, axis=0)
forecast = scaler.inverse_transform(forecast)
# Insert the last value of predictions at the 0th index of forecast
forecast = np.insert(forecast, 0, last_prediction, axis=0)
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Adjusted Close Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Adjusted Close Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Adjusted Close Prices')


data = [trace1, trace2, trace3]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)
fig.show()


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


66/66 [==============================] - 2s 10ms/step
Mean Squared Error: 0.01368112202692405
MAPE:  2.357649240831449
MAE:  0.07545289310326822
RMSE: 0.00018717309991558644
1/1 [==============================] - 0s 29ms/step


In [ ]:
# Add a dimension to the last value of predictions
last_prediction = np.expand_dims(last_prediction, axis=0)
forecast = scaler.inverse_transform(forecast)
# Insert the last value of predictions at the 0th index of forecast
forecast = np.insert(forecast, 0, last_prediction, axis=0)
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Adjusted Close Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Adjusted Close Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Adjusted Close Prices')


data = [trace1, trace2, trace3]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)
fig.show()
